In [4]:
import datetime
import pandas as pd
import numpy as np
import argparse
import sys
import string
import re
import MeCab
import mojimoji

sys.path.append('../..')
from utils.func import norm_time

In [5]:
parser = argparse.ArgumentParser()
parser.add_argument('--train_date', type=int, default=2016)
parser.add_argument('--test_date', type=int, default=2017)
# parser.add_argument('--code', type=list, default=['1711', '1712'])
# parser.add_argument('--code', type=list, default=['7203', '9984'])
# parser.add_argument('--code', type=list, 
#                     default=['8306'])

# TOPIX core30
parser.add_argument('--code', type=list, 
                    default=['2914', '3382', '4063', '4452', '4502', '4503',
                                    '4568', '6098', '6501',  '6758', '6861', '6954',
                                    '6981', '7203', '7267',  '7751', '7974', '8031',
                                    '8058', '8306', '8316',  '8411', '8766', '8802',
                                    '9020', '9022', '9432',  '9433', '9437', '9984'])
# parser.add_argument('--code', type=list, default=['8301', '7203', '9501', '6758', '9984',
#                                                  '8306', '8411', '6501', '6752', '6502'])

args = parser.parse_args(args=[])

code_list = [str(x) for x in args.code]

# テキストを取り出す

In [6]:
for i, date in enumerate(range(2013, 2020)):
    tmp = pd.read_csv('../../data/news/' + str(date) + '.csv', encoding='cp932')
    tmp = tmp[tmp['Company_IDs(TSE)'].isin(code_list)]
    # tmp = tmp[tmp['Company_Relevance'] == str(100)]
    tmp = tmp[['Time_Stamp_Original(JST)', 
                    'Company_Code(TSE)', 
                    'Headline', 
                   'Company_Name_J',
                    'News_Source',
                    'Company_Relevance', 
                    'Keyword_Article']]

    # 欠損除去
    tmp = tmp[~tmp["Keyword_Article"].isnull()]

    # タグ除去
    tmp = tmp[(tmp['News_Source'] == '日経') | 
                 (tmp['News_Source'] == 'ＮＱＮ') |
                 (tmp['News_Source'] == 'ＱＵＩＣＫ') | 
                 (tmp['News_Source'] == 'Ｒ＆Ｉ')]

    tmp['code'] = tmp['Company_Code(TSE)'].astype(int)
    tmp['date'] = pd.to_datetime(tmp["Time_Stamp_Original(JST)"]).map(norm_time)
    tmp = tmp.set_index(['date', 'code'], drop=True)
    tmp = tmp.drop(['Time_Stamp_Original(JST)', 'News_Source', 'Company_Code(TSE)'], axis=1)

    if i == 0:
        df1 = tmp.copy()
    else:
        df1 = pd.concat([df1, tmp])


# 株価を取り出す

In [101]:
for i, code in enumerate(code_list):
    tmp = pd.read_csv('../../data/stock_price/' + str(code) + '.csv', index_col=0)
    tmp['code'] = int(code)
    if i == 0:
        df2 = tmp
    else:
        df2 = pd.concat([df2, tmp])

df2['date'] = pd.to_datetime(df2['date'])
df2 = df2.set_index(['date', 'code'], drop=True)
# print(df2.shape)

df6 = pd.read_csv('../../data/nikkei_225.csv')
df6.index = pd.to_datetime(df6['date'])
df6 = df6.drop(['date'], axis=1)

df2 = df2.join(df6)

# 銘柄名を取り出す

In [102]:
comp_json = {}
with open('../../data/company_name.csv') as f:
    line = f.readline()
    while line:
        split = line[:-1].split(',')
        comp_json[split[0]] = split[1:]
        line = f.readline()

# 形態素解析

In [197]:
m = MeCab.Tagger("-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd \
                                    -u ../../mydict/mydict.dic")

# /usr/local/libexec/mecab/mecab-dict-index \
# -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd \
# -u mydict.dic \
# -f utf-8 \
# -t utf-8 \
# mydict.csv

In [198]:
def Morph(row):
    comp_name = comp_json[str(row.name[1])]
    text = row['Headline']
    output_words = []

    # 前処理
    text = re.sub("(\*J)",  "", text)
    text = re.sub(" |　", "", text)
    text = mojimoji.zen_to_han(text, kana=False, digit=True)
    text = re.sub(r"\\u3000",  "", repr(text)[1:-1])
    text = re.sub(r"<.+>",  "", text)
    text = re.sub(r"＜.+＞",  "", text)
    text = re.sub(r"\(\d\)",  "", text)
    text = re.sub(r"|\.|（|）|「|」|｢|｣|―|：|【|】|~|\(|\)|。|/|:|\[|\]|,|\*|◇|☆|◎|;|&",  "", text)
    text = re.sub(r"\d+",  "0", text)

#     for p in string.punctuation:
#         text = text.replace(p, ' ')
    
    # company
    for name in comp_name:
        text = text.replace(name, '<company>')
    
    soup = m.parse(text)
    for i, row in enumerate(soup.split('\n')):
        category = row.split('\t')
        if category[0] == "EOS":
            break
        elif (i == 0) and (category[0]  =='人事'):
            break
        else:
            
            word = category[0]            
            tag = category[1].split(',')
            if tag[2] == '人名':
                word = '<person>'
            elif tag[2] == '地域':
                word = '<location>'
            elif tag[2] == '組織':
                if (category[0] != '東証') and (category[0] != '時価総額'):
                    word = '<organization>'
            elif (tag[0] == '助詞') or (word=='、') or \
                    (word=='､') or (word=='・') or (word=='･') or (word == ''):
                continue

            output_words.append(word)
    
    if len(output_words) < 4:
        return None
    else:
        return ':'.join(output_words)

In [199]:
arr1 = df1[0:2000].apply(Morph, axis=1)
arr2 = df1[0:2000]['Headline']

In [200]:
for x1,x2 in zip(arr1, arr2):
    print(x1)
    print(x2)
    print()

<organization>:米子会社:<location>:コンサル:会社:買収
<日経>◇日立の米子会社、英コンサル会社を買収

<company>:スマホ:また:通信:障害:データ:通信:でき:ず
<日経>◇ａｕスマホ、また通信障害　データ通信できず

<company>:全:グループ:海外:研修:将来:幹部:育成
<日経>◇三菱商事、全グループで海外研修　将来の幹部育成

<person>:<location>:知事:企業:立地:着実:自動車:0:税:撤廃:要請
<日経>◇大村・愛知知事「企業立地、着実に」　自動車２税撤廃要請

Uターンラッシュ:鉄道:空の便:とも:ピーク
<日経>◇Ｕターンラッシュ　鉄道・空の便ともにピーク

0年:<organization>:優秀:製品:サービス:賞:新た:な:着想:個性:光る
<日経>◇2012年日経優秀製品・サービス賞　新たな着想、個性光る

<company>:証券:社:<organization>:信:証:連携:中心:<company>:インベ:合併
<ＮＱＮ>◇みずほ証券社長「銀信証の連携の中心に」　みずほインベと合併

日経平均:大幅続伸:始まる:崖:回避:円安:好感:し:震災前:水準:上回る
<ＮＱＮ>◇日経平均、大幅続伸で始まる 「崖」回避と円安を好感し震災前水準上回る

日経平均:大幅続伸:始まる:崖:回避:円安:好感
<日経>◇日経平均、大幅続伸で始まる 「崖」回避と円安を好感

<location>:先物取引:委:<location>:<company>:証券:民事:制裁金:支払い:命令
<日経>◇米先物取引委、米国みずほ証券に民事制裁金支払い命令

<location>:<company>:証券:0万:円:罰金:<organization>:顧客:資産:管理:巡り
<ＮＱＮ>◇米国みずほ証券に1500万円罰金　ＣＦＴＣ、顧客資産の管理巡り

<company>:米:法人:機器:間:無線通信:合弁:設立
<日経>◇ソニー米法人、機器間無線通信で合弁設立

東証:0部:時価総額:0兆:円:上回る:0年0月:以来
<ＮＱＮ>◇東証１部の時価総額、300兆円上回る　11年３月以来

<company>:一時:0%:超:高:円安:進行:一段:採算:改善:期待
<ＮＱＮ>◇＜東証＞トヨタが一時５％超高　円安


<company>:一段:高:含み益:増:リニア:期待
<NQN>◇＜東証＞ＪＲ東海が一段高　含み益増、リニアに期待も

None
<日経>◇人事、三菱商事ＲｔＭジャパン

None
<日経>◇人事、三菱商事ＲｔＭジャパン

None
<日経>◇人事、リクルートホールディングス

None
<日経>◇人事、リクルートホールディングス

<company>:会長:<person>:<person>:<person>:会長:名誉会長:就任
<NQN>◇トヨタ会長に内山田副会長　張会長は名誉会長に就任

0%:ﾙｰﾙ:報告:0日:<company>:0:保有:割合:減:三井住友信託銀行株式会社
<QUICK>5%ﾙｰﾙ報告６日 三菱商(8058)―保有割合の減(三井住友信託銀行株式会社など)

0%:ﾙｰﾙ:報告:0日:<company>:0:保有:割合:減:三井住友信託銀行株式会社
<QUICK>5%ﾙｰﾙ報告６日 三菱UFJ(8306)―保有割合の減(三井住友信託銀行株式会社など)

<company>:社外取締役:制度:導入:<organization>:相談役:元:GM:副社長
<NQN>◇トヨタ、社外取締役制度を導入　日生相談役や元ＧＭ副社長

<company>:自動車:事業:0:つ:ビジネス:ユニット:設置
<NQN>◇トヨタ、自動車事業の４つのビジネスユニット設置

<company>:社長:新体制:中期:計画:実現:向け:前進:する:ため
<NQN>◇トヨタ社長、新体制「中期計画の実現に向け前進するため」

<company>:社長:国内:生産:0万:台:維持
<NQN>◇トヨタ社長「国内生産300万台は維持」

<company>:<location>:<location>:石化:製品:貯蔵:基地:0年:春:操業:開始
<日経>◇三井物産、米ヒューストンに石化製品貯蔵基地　15年春操業開始

None
<日経>◇人事、トヨタ自動車

None
<日経>◇人事、トヨタ自動車

None
<日経>◇人事、トヨタ自動車

None
<日経>◇人事、トヨタ自動車

<organization>:iPhone:0:S:販売:<company>:代行
<日経>◇ウィルコム、ｉＰｈｏｎｅ４Ｓを販売　ソフトバンクの代行

株価:格付:SMBC:<organizati

In [201]:
df4 = df1.copy()
df4['Headline'] = df1.apply(Morph, axis=1)
df4 = df4.dropna(subset=['Headline'])

In [202]:
m.parse('CEO')

'CEO\t名詞,固有名詞,一般,*,*,*,*,シーイーオー,シーイーオー,シーイーオー\nEOS\n'

# テキストと株価をくっつける

In [203]:
# 時系列をくっつける
df3 = pd.concat([df4, df2], axis=1, join_axes=[df4.index], levels=[0, 1])
df3 = df3.dropna()
df3 = df3.sort_values(by=['code', 'date'])
df3['Headline'] = \
    df3.groupby(level=[0,1]).apply(lambda x: ':<span>:'.join(list(x['Headline'])))

df3 = df3.dropna()
df3 = df3[~df3.duplicated(subset=['Headline'])]
df3['price'] = \
        df3['adj_close'].groupby(level=['code']).pct_change(1).shift(-1)*100
df3 = df3.dropna()

/home/shogo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  


In [204]:
def calc_act_return(x):
    x = x.pct_change(1).shift(-1)*100
    return x['adj_close'] - x['value']

In [205]:
# 時系列をくっつける
df3 = pd.concat([df4, df2], axis=1, join_axes=[df4.index], levels=[0, 1])
df3 = df3.dropna()
df3 = df3.sort_values(by=['code', 'date'])
df3['Headline'] = \
    df3.groupby(level=[0,1]).apply(lambda x: ':<span>:'.join(list(x['Headline'])))

df3 = df3.dropna()
df3 = df3[~df3.duplicated(subset=['Headline'])]
df3['price'] = \
        df3['adj_close'].groupby(level=['code']).pct_change(1).shift(-1)*100
df3['active_return'] = \
    df3[['adj_close', 'value']].groupby(level=['code']).apply(calc_act_return).values
df3 = df3.dropna()

/home/shogo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  


In [206]:
df1[df1['Company_Name_J'] == 'ソフトバンクグループ'].iloc[-40]

Headline                               <日経>◇ＴモバＣＥＯ、スプリント合併へ「承認獲得は最終段階」
Company_Name_J                                              ソフトバンクグループ
Company_Relevance                                                   50
Keyword_Article      ニューヨーク:携帯電話:経営統合:目指す:携帯:６月:当局:承認:獲得:見通し:維持:合併:...
Name: (2019-04-26 00:00:00, 9984), dtype: object

In [207]:
df3[df3['price'] * df3['active_return'] < 0].iloc[-1]['Headline']

'スタートアップ:進める:オフィス:改革:<span>:スタートアップ:進める:オフィス:改革:<span>:<organization>:モバ:CEO:<organization>:合併:承認:獲得:最終:段階'

In [208]:
print(df3['price'].shape)
print(df3['price'][np.abs(df3['price']) > 1].shape)

(11725,)
(7380,)


In [209]:
# CSVファイルに保存する
df5 = pd.concat([df3[['Headline', 'active_return']].rename(
                                   columns={'Headline': 'state', 'active_return': 'reward'}),
                             df3[['Headline']].shift(-1).rename(
                                   columns={'Headline': 'next_state'})], axis=1).dropna()
df5 = df5[['state', 'next_state', 'reward']]

date_year = df5.index.map(lambda x: x[0].year)

df5[date_year <= args.train_date].to_csv(
    '../../data/news/text_train.tsv',
    header=None,
#     index=None,
    sep='\t')

df5[(args.train_date < date_year) & (date_year < args.test_date)].to_csv(
    '../../data/news/text_val.tsv',
    header=None,
    index=None,
    sep='\t')

df5[(args.test_date <= date_year)].to_csv(
    '../../data/news/text_test.tsv',
    header=None,
#     index=None,
    sep='\t')

# df5[(args.test_date <= date_year) & (date_year <= 2017)].to_csv(
#     '../../data/news/text_test.tsv',
#     header=None,
#     index=None,
#     sep='\t')

# 銘柄の名前リストを入れる

In [210]:
# for i, date in enumerate(range(2011, 2020)):
#     tmp = pd.read_csv('../../data/news/' + str(date) + '.csv', encoding='cp932')
# #     tmp = tmp[tmp['Company_IDs(TSE)'].isin(code_list)]
#     # tmp = tmp[tmp['Company_Relevance'] == str(100)]
#     tmp = tmp[['Time_Stamp_Original(JST)', 
#                     'Company_Code(TSE)', 
#                     'Headline', 
#                    'Company_Name_J',
#                     'News_Source',
#                     'Company_Relevance', 
#                     'Keyword_Article']]

#     # 欠損除去
#     tmp = tmp[~tmp["Keyword_Article"].isnull()]

#     # タグ除去
#     tmp = tmp[(tmp['News_Source'] == '日経') | 
#                  (tmp['News_Source'] == 'ＮＱＮ') |
#                  (tmp['News_Source'] == 'ＱＵＩＣＫ') | 
#                  (tmp['News_Source'] == 'Ｒ＆Ｉ')]

# #     tmp['code'] = tmp['Company_Code(TSE)'].astype(int)
# #     tmp = tmp.set_index(['date', 'code'], drop=True)
#     tmp = tmp.drop(['Time_Stamp_Original(JST)', 'News_Source'], axis=1)

#     if i == 0:
#         df5 = tmp.copy()
#     else:
#         df5 = pd.concat([df5, tmp])

In [19]:
# df5 = df5[['Company_Code(TSE)', 'Company_Name_J']].dropna()
# df5['Company_Code(TSE)'] = df5['Company_Code(TSE)'].astype(int)
# df5 = df5.sort_values('Company_Code(TSE)')

In [20]:
# df5[~df5.duplicated()].to_csv('../../data/company_name.csv', index=False, header=False)